In [1]:
import json
import boto3
import pandas as pd
from io import BytesIO

In [2]:
# Access aws credentials from json file
aws_creds_json = "aws_credentials.json"
with open("aws_credentials.json", 'r') as file:
    aws_creds_json = json.load(file)
# Store necessary authentication in dictionary
aws_creds = {
    'service_name': 's3',
    'aws_access_key_id': aws_creds_json["aws_access_key_id"],
    'aws_secret_access_key': aws_creds_json["aws_secret_access_key"],
    'region_name': 'eu-central-1'
}
# Specify the s3 bucket name we work with
bucket_name = 'fs-reghub-news-analysis'

In [3]:
# Connect to aws using two methods
s3c = boto3.client(**aws_creds)
s3r = boto3.resource(**aws_creds)

In [4]:
# Show the files in the s3 bucket
bucket = s3r.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(obj.key)

categories.csv
data_rule_labels_updated_rough_cleaning.csv


In [5]:
# Example object key
object_key = 'data_rule_labels_updated_rough_cleaning.csv'
# Example output key (Only needed for download(Will be the new file name))
#output = 'data_rule_labels_updated_rough_cleaning.csv'

In [6]:
# Dowload the file if needed
#s3c.download_file(bucket_name, object_key, output)

In [7]:
# Directly store the file in a pandas df
# Access the object on aws s3
response = s3c.get_object(Bucket=bucket_name, Key=object_key)
# Read the file content
file_content = response['Body'].read()
# Convert to csv-like
csv_object = BytesIO(file_content)
# Store as a pandas df for further use
df = pd.read_csv(csv_object)

df.head(2)

,Unnamed: 0,_id,_index,_score,_type,concepts,found_dates,id,internal_links,keywords,...,news_title,news_topic,news_type,references,sentiment_label,sentiment_score,source_country,source_name,source_type,rule_labels_comb
0,0,fin1023Preslofreshbuy917302,2021_reghub_new,NaN,_doc,NaN,[],fin1023Preslofreshbuy917302,[],NaN,...,HelloFresh Buy von Joh. Berenberg Gossler & Co...,"[{""description"":""Berenberg Bank"",""relevance"":4...",Press releases,[],NaN,0.0,Global,finanzen.at,Information Provider,['market']
1,1,finichtenStoceaaufbuy322htm,2021_reghub_new,NaN,_doc,NaN,[],finichtenStoceaaufbuy322htm,[],NaN,...,BERENBERG stuft Nordea auf 'Buy' Finanznachri...,"[{""description"":""Berenberg Bank"",""relevance"":6...",Stock Analysis,[],NaN,0.0,Global,finanznachrichten.de,Information Provider,[]


In [8]:
# Upload a file to aws
# Specify path
example_path = "../categories.csv"
# Desired Name (If already on aws s3, the file will be overwritten)
new_object_key = "categories.csv"
# Upload the file
s3c.upload_file(example_path, bucket_name, new_object_key)